### DATASCI 2000 COURSE NOTES


# Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.optimize as so
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline

# Read DF 
df = pd.read_csv("run10sample.csv")

In [ ]:
.mean() # Takes mean
.std() # Takes standard deviation 
.value_counts()["x"] # Number of x in a category ex. nMenS = df['gender'].value_counts()['M']

## Lecture 1

## Lecture 2

Contingency Tables: summarizes data for two categorical variables

In [ ]:
table = pd.crosstab(df['row'], df['column'], margins = True)
# add normalize = "index" for realtive ratios

# Index using 

table.iloc['row']['column']

# Table based on a certain condition 

table2 = pd.crosstab(df["genre"] == "comedy", df["dirGender"], margins = True)


### Graph Set up - General Examples

In [ ]:
# One Graph 
fig, axes = plt.subplots(figsize = (10,5))
sns.violinplot(data = df, y = "income", x = "gender", ax = axes, color = "y")
axes.set_ylabel("Income")
axes.set_xlabel("Gender")


# Multiple graphs
x = np.arange(len(table.columns))
width = 0.4

fig, axes = plt.subplots(2, figsize = (10, 10))
axes[0].bar(x - width/2, table.iloc[0], width, label = "female")
axes[0].bar(x + width/2, table.iloc[1], width, label = "male")

axes[0].legend()
axes[0].set_xticks(x, table.columns)
axes[0].set_ylabel("Number of People")
axes[0].set_xlabel("Workclass")


axes[1].bar(table.columns, table.iloc[0], width, label = "female")
axes[1].bar(table.columns, table.iloc[1], width, bottom = table.iloc[0], label = "male")
axes[1].set_ylabel("Number of People")
axes[1].set_xlabel("Workclass")
fig.tight_layout()

Barplots: way to display a single categorical variable 
Realitive frequency bar plot: proportions instead of frequencies are shown 

Barplot vs histogram?
Bar plots are used for displaying distributions of catgorical variable, while histograms are used for numerical variables 

In [ ]:
sns.barplot(data = df, x = 'day',y='bikecount', hue='place')
# Grouped Bar Chart
axes[0].bar(x - width/2, table.iloc[0], width, label = "female")
axes[0].bar(x + width/2, table.iloc[1], width, label = "male")

# Stacked Bar Chart
axes[1].bar(table.columns, table.iloc[0], width, label = "female")
axes[1].bar(table.columns, table.iloc[1], width, bottom = table.iloc[0], label = "male")

g = table.plot(kind = "bar", stacked = True)

# Histogram
sns.histplot(data = df, x = incomeMales, ax = axes, color = "b", bins = 100, alpha = 0.3)
axes[0,1].hist(df['educational-num'], bins = 16, color = 'g')

# Box Plot 
sns.boxplot(data = df, y = "income", x = "gender", ax = axes, color = "b")

# Violin Plot
sns.violinplot(data = df, y = "income", x = "gender", ax = axes, color = "y")
sns.violinplot(data = df, y = "income", x = "workclass", hue = "gender", ax = axes, color = "r")


## Probabilities

General **Addition rule**: P(A or B) = P(A) + P(B) - P(A and B)

**Disjoint Events:** Events that are mutally exclusive, if one happens the other can't  
--> P(A or B) = P(A) + P(B)

**Marginal Probabilty:** The probability of an event occuring   
(total of a category / total of all)  ==> P(A)

**Joint Probability:** The likelihood of two events occurring together   
(Crosstable value of 2 events / total of all)  ==> P(A and B)

**Conditional Probability:** Outcome of intrest A given condition B  
-- What you're given is the row/colum you look at  
(crosstable value of 2 events / total of all) / (total of a category / total of all)  
*(crosstable value of 2 events / total of a category)  ==> P(A|B) = P(A and B) / P(B)*

**Multiplication Rule**: P(A and B) = P(A|B) * P(B) (OR P(B|A) * P(A))
Condition probability rearranged  
Useful to think of A as the outcome of interest and B as the condition

**Independence:** Giving B doesn't say anything about A  
--> P(A|B) = P(A)  
If A and B are independent: P(A and B) = P(A) * P(B)  
--> Joint is product of marginals

Bayes Rule: P(A|B) = ( P(B|A) * P(A) ) / P(B)


Inverting probabilities: See lecture 2 slides for tree breakdown + note on bayes theorem 



### Grouping

In [ ]:
incomeMales = df['income'].groupby(df['gender']).get_group("Male")
meanIncomeMales = incomeMales.mean()

incomeFemale = df['income'].groupby(df['gender']).get_group("Female")
meanIncomeFemales = incomeFemale.mean()


menFinishingTimeS = df["time"].groupby(df["gender"]).get_group("M").mean()

womenFinishingTimeS = df["time"].groupby(df["gender"]).get_group("F").mean()

## Lecture 3

**Confidence interval:** A plausible range of values for the population parameter  
The width of the interval contains xx% of possible population means for a given sample mean 
- A wider interval allows us to be more certain that we capture the population parameter but is not as informative  

**Bootstrap:** Resampling (with replacement) from the sample
- The distribution of bootstap stats approximate the distribution of the sample stats
Pros:
- Unversal technique to obtain confidence intervals
- Can be applied to any statistics
- Confidence intervals are asymptotically correct
- Does not make assumptions about the underlying distribution 
Cons:
- Requires programmign skill
- Can become noisy with small N 
- Takes computation time


In [ ]:
def confidenceInt(data, prec):
    bounds = []
    bounds.append(np.percentile(data, 100-prec))
    bounds.append(np.percentile(data, prec))
    
    return bounds

# Input arguments 
# - data: data series to resample
# - N: Sample size for each iteration 
# - fcn: function to apply to the sample to get the statistics
# - numIter: Number of resamples (should default to 1000) 

# Output argument: 
# - Numpy array of size numIter that contains the estimates of the statistics (i.e. the bootstrap sample)

def bootstrap(data, N, fcn, numIter = 1000):
    sample = np.array([])
    
    for i in range (0, numIter):
        value = np.random.choice(data, replace = True, size = N)
       
        x = fcn(value)
        
        sample = np.append(sample, x)
        
    return sample

In [ ]:
# Graph with bootstrap and confidence int

fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (20,7))
axes.hist(bootSamp, bins = 30, color = 'b')
axes.set_xlabel("Mean Time")
axes.set_ylabel("Frequency")
plt.axvline(x = confidenceInt(bootSamp, 95)[0], color = 'r')
plt.axvline(x = confidenceInt(bootSamp, 95)[1], color = 'r')

**Standard Error of the mean**: the standard deviation of a set of sample means
- a measure of the expected variability of our estimates
- When sample size (N) quadrupels, SEM halves  
SEM = SD(x) / sqrt(N) (population SD / Sqrt of sample size)

The **standard error of means** nearly matches the **standard deviation of the means**. Therefore the assumption is that the standard error of the population can be estimated from the standared deviation of the mean of distributions.

We can't get a confidence interval from the SEM alone we alone we also need shape

**Central Limit Theorem:** The sampling distribution of means based on large sample size, can be approximated by a normal distribution 
- If you sum a large nymber of random variables (of any distribution), the sum will have a normal distribution

Normal model:

 $$\bar{x} \sim N(mean = \mu, SE = \sigma / \sqrt{n})$$
 
 Where SE is the standard error, which is the SD of the sampling distribution
 
Conditions for CLT:
Independence: Sampled observations must be independent, difficult to verify but is more likely if
- Random sampling/assignment is used
- if sampling without replacement, n < 10% of the population  
Sample Size/Skew: EIther the population distribution is normal, or if the population distribution is skew, the smaple size is large
- the more skewed the population distribution, the larger sample size we need for CLT to apply
- for moderately skewed distributions n > 30 is a widely used rule of thumb


## Lecture 4

Explanatory (independent) variables: indicate conditions we can impose on the experimental units  
Blocking variables: characteristics that the experimental units come with, that we would like to control for
- like stratifying, except used in experimental settings when randomly assigning, as opposed to when sampling   

Confounding variable: variable that co-varies with the independent varibale, and which can provide an alternative explination for the experimental effect  

Placebo: fake treatment, often used as the control group for medical studies

Placebo effect: experimental units showing improvement simply beacsue they believe they are receiving a special treatment  

Blinding: when experimental units do not know whether they are in the control or treatment group 

Double-blind: when both the experimental units and the reasearchers who interact with the patients do not know who is in the control and who is in the treatment group

Null hypothesis: There is nothing going on (x is independent)  
Alternative hypothesis: There is something going on (x is dependent)  

Randomization methods:
- **Randomization/Permutation Test:** Redo random assignemnt performed in the experiment K times or Check all possible ways of the random assignment
- **Monte-Carlo Simulation:** generate new data under a parametric model for the Null-Hypothesis 
- **Bootstrap:** use our sample as a model for the population and draw K new samples of size N 


In [ ]:
# Randomization / Permutation

def permutation_test(frame, fnc, shuffle, numIter = 500, sides = 1, pBin = 25):
    sample = np.array([])
    
    for i in range(0, numIter):
        df_copy = frame.copy()
        idx = df_copy.index
        idx_arr = np.arange(0, len(idx))
        np.random.shuffle(idx_arr)
        shuffled_gender = df_copy[shuffle].iloc[idx_arr]
        shuffled_gender_id = shuffled_gender.reset_index(drop = True)
        df_copy[shuffle] = shuffled_gender_id
        sample = np.append(sample, fnc(df_copy))
        
    # Graph     
    fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (20,10))
    axes.hist(sample, bins = pBin, color = 'b')
    plt.axvline(x = fnc(frame), color = 'r')
    axes.set_ylabel("Number of Observations")
    axes.set_xlabel(f"Result of test")
    
    # P-Value
    if sides == 2:
        plt.axvline(x = -(fnc(frame)), color = 'r')
        a = np.absolute(sample) >= fnc(frame)
        twosided = sum(a) / len(sample)
    
        print(f"The result of the two sided test is {twosided}")
        
    else:
        a = sample >= fnc(frame)
        p = sum(a) / len(sample)
        print(f"The p-value is {p}")
    
    return sample

**Errors:**
- **Type 1 Error** is rejecting the null hypothesis when H0 is true. 
- The *probability of the the type one error is the p-value* p(stats >thres|H0)
- If making a Type 1 error is dangerous of especially costly, we should choose a small significance level (ex. 0.01) 
    - We want to be very cautious about rejcting the nul hypothesis, so we demand very strong evidence favouring HA before we would reject H0



- **Type 2 Error** is failing to reject the null hypothesis when HA is true
- The p-value does not tell us anything about this 
- If making a type 2 error is relatively more dangerous or much more costly than a type 1 error than we should choose a higher significance level (ex. 0.10) 
    - We want to be cautious about failing to reject H0 when the null is actually false.

## Lecture 5

**Chi-Square Test of Independence:** the X2 test statistic measures the deviation between observed counts for each cell (O) and the expected counts (E) under the null hypothesis

$$ X2 =  \sum_{i} {(O_i - E_i)^2 \over E_i }$$

Expected count(A and B) = P(A) * P(B) / table total 

We have a test statistic that quantifies how far our data deviates from the prediction of independence (larger statistic = larger deviation)

In [ ]:
# Chi-Squared

def X2(data, x = "genre", y = "dirGender"):
    table = pd.crosstab(data[x], data[y])
    tabArr = np.array(table)
    
    nrow, ncol = tabArr.shape
    expectArr = np.zeros([nrow, ncol])
    
    sObservedCol = np.sum(tabArr, axis = 0)
    sObservedRow = np.sum(tabArr, axis = 1)

    for i in range(0, len(sObservedCol)):

        marginalC = sObservedCol[i]

        for j in range (0, len(sObservedRow)):

            marginalH = sObservedRow[j]

            expectArr[j,i] = (marginalH * marginalC) / np.sum(tabArr)
            
    X2 = np.sum((tabArr - expectArr)**2/expectArr)
    return X2

**Monte-Carlo Test:**
- Simulation done under the null hypothesis
- Count how many cases a value equal or smaller than the measured value is produced 

In [ ]:
# Monte-Carlo Test

# prob = probability of a success on each coin toss
# N = the number of coin tosses per trial (N)
# t = the number of trials

# Simulates COIN TOSSES
def monteCarloSim(prob, N, t):

    numHead = np.array([])

    for trial in range(0, t):
        sim = np.random.choice([0,1], size = N, replace = True, p = prob) # p is probability per element
        numHead = np.append(numHead, np.sum(sim))
    
    return numHead


# H0prob = The probability of heads under the Null hypothesis
# N = The number of coin throws per trial 
# numHeads = The number of observed coin tosses. 

# Generates GRAPH
def monteCarloTest(H0prob, N, numHeads):
    data = monteCarloSim(H0prob, N, 2000)
    
    fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (20,10))
    axes.hist(data, bins = 15, color = 'b')
    plt.axvline(x = numHeads, color = 'r')
    axes.set_ylabel("Number of Observations")
    axes.set_xlabel(f"Number of Complications")
    
    a = data <= numHeads
    p = sum(a) / len(data)
    print(f"The p-value is {p}")
    
    return data


**Power:** the probability that the test correctly rejects the null hypothesis when a specific alternative hypothesis is true
- Need to simulate data under the HA
- Need to specify the HA precisely
- The difference between the H0 and the HA is called the **effect size**
- As power decreases type 1 error decreases and type 2 error increases 
- Tells us whether we had a good chance of detecting an effect of a specific size
- If we are under-powered we can: 
    - Decrease our threshold and accept a higher type 1 error
    - Increase our sample to be appropriately powered
    
**P-Value is the probability of rejecting H0 if the H0 is true  
Power is the probability of rejecting the H0 is the HA is true**

## Lecture 6

**Bayesian Inference:** does not use a decision criterion. Only care about the data we observed.
- The prior makes inderence subjective
- The evidence can be seen by two different people and the posteriors can be very different
- While that is reflecting what truly happens, it is not desirable for reporting results objectively 

**Bayes Factor**:Tells you how much you need to update your prior belief about the probabilities, so it quantifies the evidence independent of priors  
Can be interpreted as a betting ratio that would give you a fair bet  
Bayesian updating: how to update the ratio of your beliefs

$$ BF_{10} = { p(Data|H_A) \over p(Data|H_0)}$$

**Bayesian hypothesis testing:** 
- Treats H0 and HA the same way
   - if BF10 is < 1, look at BF01 = 1/BF10
- All hypotheses need to be fully specified
- Conclusions depend on the prior probability with which each hypotheses is true

| Bayes Factor        | Evidence         |
|---------|--------------------|
| <1    | Negative, in favour of the other hypothesis |
| 1-3 | Barely worth mentioning |
| 3-20     | Positive              | 
| 20-150| Strong|
|>150| Very Strong|  

Bayesian Inference for continous variables:
- Do a randomization test to get the SD of the statistic under the H0 
- Assume that p(stats|H0) is noraml (0, SD)
- Assume that p(ststs|HA) is normal(effect size, SD)
- Look up the corresponding probability densities on using ss.normal.pdf

In [ ]:
def bayesFactor(probH0, probH1, N, t, obs, prior):

    H0numHead = np.array([])
    H1numHead = np.array([])

    # Coin Tosses
    for trial in range(0, t):
        H0sim = np.random.choice([0,1], size = N, replace = True, p = [1-probH0, probH0]) # p is probability per element
        H0numHead = np.append(H0numHead, np.sum(H0sim))
        
        H1sim = np.random.choice([0,1], size = N, replace = True, p = [1-probH1, probH1]) # p is probability per element
        H1numHead = np.append(H1numHead, np.sum(H1sim))
        
#         H0numHead = np.append(H0numHead, np.random.binomial(N, probH0))
#         H1numHead = np.append(H1numHead, np.random.binomial(N, probH1))
      
    # Graphs 
    ax = sns.distplot(H0numHead, color = 'r')
    sns.distplot(H1numHead, ax = ax, color = 'b')
    plt.axvline(x = obs, color = "black")

    # P-Value 
    #Probability of rejecting H0 if H0 is true
    
    pValue = sum(H0numHead < obs)/len(H0numHead)
    print(f"The p-value is {pValue}")
    
    if pValue <= 0.05:
        print("The null hypothesis was rejected due to the low p-value")
    else:
        print("There was not sufficient evidence found to reject the null hypothesis")
    
    # Power
    # prbability of rejectign H0 if HA is true
    
    power = sum(H1numHead < obs) / len(H1numHead)
    print(f"The power is {power}")
    
    if power >= 0.80:
        print("The null hypothesis was rejected due to the high p-value")
    else:
        print("There was not sufficient evidence found to reject the null hypothesis")
    
    
    # Bayes Factor
    # BF10 = p(Data|H1)/p(Data|H0)
    

    # Read from the alternative distribution 
    BF1 = sum(H1numHead == obs)/len(H1numHead)

    # p(observign 8 side effects given | H0)

    # Read from the null distribution
    BF0 = sum(H0numHead == obs)/len(H0numHead)

    BF10 = BF1/BF0

    print(f"\nThe bayes factor is {BF10}")
    
    
    # Posterior Probability
    
    prior0 = prior
    prior1 = 1 - prior
    posterior1 = (prior1*BF1)/(prior1*BF1 + prior0*BF0)
    print(f"The prosterior probability of the alternative hypothesis is {posterior1}")
    
    

# Continous Variable Bayesian 
p0 = ss.norm.pdf(meanDiff(df), 0, STD)

p1 = ss.norm.pdf(meanDiff(df), effect, STD)

# effect = 0.5?

# Calculate BF10
BF10 = p1/p0
print(BF10)


Posterior Probability: the revised or updated probability of an event occurring after taking into consideration new information

## Lecture 7

### Simple Linear Regression
A simple linear regression model follow y = mx + b or y = b0 + b1x  

Loss funtion contains the criterion of what consitutes a good and bad fit (RSS = loss)
- High loss = bad fit
- Low loss = good fit
- We aim to pick parameters than minimize loss  
If intercept and slope are negative, increase values to get the lowest rss (closest to 0)
If intercept and slope are positive, decrease values to get the lowest rss (closest to 0)

**Residuals**: are the leftovers from the model (R = y_act - y_pred)
- Each data point gives a residual 

We aim for a line that has small residuals 
- **Option 1**: Minimize the sum of the absolute value of residuals
- **Option 2**: Minimize the sum of squared residuals  

Least sqaured is more commonly used for a few reasons
1. Motivated by normal distribution of errors
2. Solutions can be easily computed
3. Big errors count relatively more than small errors

In linear regression we have two parameters making the loss-function a surface.  
We can speed up fitting a lot if we provide the derivitive of the parameters.
##### ADD MATH TO PAPER CHEAT CHEET

The vector of partial derivatives is called a gradient our loss function returns the rss (loss) adn the gradient (b values)

Quality of fit is assessed using R2, also called the coefficient of determination.  
R2 is calculated from the ration of rss to tss.  
It tells us what percent of variability in the response variable is explained by the model (0 = no fit, 1 = perfect fit)  
The remainder of the variability is explained by variables not incuded in the model or by inherent randomness in the data  

**Outliers**: points that lie away from the cloud of points  
Outliers that lie horizontally away from the center of the cloud are called high or lowleverage points. These points may influence the slope of the regression line and are called influential points.


In [ ]:
def simpleRegPredict(b, x):
    yPredicted = b[0] + b[1]*x
    return yPredicted

def plotPrediction(b,x,y,fcn = simpleRegPredict):
    plt.scatter(x, y) 
    xPredict = np.linspace(min(x), max(x), num = len(x))
    plt.plot(xPredict, fcn(b, xPredict), color = 'red')

def simpleRegLossRSS(b, x, y):
    yPredict = simpleRegPredict(b, x)

    # calculate the residuals (difference between the real and predicted y values)
    res = y - yPredict
    rss = res**2
    loss = np.sum(rss)

    # calculate the derivatives with respect to each parameter (regression coefficient)
    db0 = np.sum(-2*(res))
    db1 = np.sum(-2*x*res)
    
    #dloss/db
    deriv = [db0, db1]
    
    return loss, deriv

def simpleRegFit(x, y):
    b0 = [0,0]
    
    result = so.minimize(simpleRegLossRSS, b0, args=(x, y), jac=True)
    
    b = result.x
    plotPrediction(b, x, y, fcn = simpleRegPredict)
    
    TSS = sum((y - np.mean(y))**2)
    
    yPredict = b[0] + b[1]*x
    res = y - yPredict
    rss = res**2
    loss = np.sum(rss)
    
    R2 = 1 - (loss/TSS)

    return R2, b

def simpleRegFit(x, y, fcn = simpleRegLossRSS):
    b0 = [0,0]
    
    result = so.minimize(fcn, b0, args=(x, y), jac=True)
    
    b = result.x
    plotPrediction(b, x, y, fcn = simpleRegPredict)
    
    TSS = sum((y - np.mean(y))**2)
    
    loss, d = fcn(b,x,y)
    
    R2 = 1 - (loss/TSS)

    return R2, b


#OUTLIER EXCLUSION - Likely not needed 
df2 = df

q75 = np.percentile(df2['tailL'], 75, interpolation = 'midpoint')
q25 = np.percentile(df2['tailL'], 25, interpolation = 'midpoint')

IQR = q75 - q25
upper = q75 +1.5*IQR
lower = q25 - 1.5*IQR

upper_array=np.array(df2['tailL'] >= upper)

b = list(upper_array)

lower_array=np.array(df2['tailL'] <= lower)

c = list(lower_array)

for i in range(len(b)):
    if b[i] == True:
        df2.drop(i, inplace = True)
        
for i in range(len(c)):
    if c[i] == True:
        df2.drop(i, inplace = True)

## Lecture 8

In [ ]:
def simpleRegLossSAD(b, x, y):
    
    yPredict = simpleRegPredict(b, x)

    # calculate the residuals (absolute difference between the real and predicted y values)
    res = y - yPredict
    rss = abs(res)
    loss = np.sum(rss)

    # calculate the derivatives with respect to each parameter (regression coefficient)
    db0 = np.sum(-2*(res))
    db1 = np.sum(-2*x*res)
    
    #dloss/db
    deriv = [db0, db1]
    
    return loss, deriv

### Polynomial regression and Cross Validation 
Robust statistics aim to provide methods that emulate popular statistical models but are not unduly affected by outliers or other small departures from model assumptions
- **Mean** is a measure of central tendency that is sensistve to outliers / strong skew
- **Median** is a measure of central tendency that is robust against outliers

With each polynomial term, we fit the data better. Extrapolation of the fit can change dramatically but you lose the extrapolation ability.  
Models with more free parameters are more complex models.  
Complex models (usually) fit the data better than simple models (every model is nested in the previous.  
Fit is therefore not a good criterion to compare models.  
Good models should *predict* new data better.

A predictive R2 tells us which proportion of the variance of y can be predicted (rather than fitted).
Can be lower than 0, this means a simple mean of the data is a better predictor.  
Models of different complexity can be directly compared. The models with ethe best predictive R2 is makeing the best predictions and can be consisdered the most appropriate description of the data

Types of Cross validation:
- Leave one out: leave each data point out as the test set and use all other points for training
- Split half: use half the data set as test set and the other half as training. Then reverse the assignemnt
- K-fold: split the data into k parts, each time leave one part out as test set and train on the others.
    - K = 2... Split half
    - K = N... Leave one out 



In [ ]:
def polyRegPredict(b, x):
    
    yp = np.zeros(x.shape)

    for i in np.arange(len(b)): 
        yp = yp + (b[i]*(x**i))
            
    return yp

def polyRegLossRSS(b, x, y):
    
    yPredict = polyRegPredict(b, x)

    # calculate the residuals (difference between the real and predicted y values)
    res = y - yPredict
    rss = res**2
    loss = np.sum(rss)

    # calculate the derivatives with respect to each parameter (regression coefficient)
    
    deriv = []
    for i in range(len(b)):
        deriv.append(-2 * sum((y - polyRegPredict(b, x)) * x**i))
    
    return loss, deriv

def polyRegFit(x, y, order, fcn, fig = True):
    # Order 0 = order 1... therefore order = order requested + 1
    
    result = so.minimize(fcn, np.array([0 for i in range(order)]), args=(x, y), jac=True)
    
    b = result.x
    if fig == True:
        plotPrediction(b, x, y, fcn = polyRegPredict)
    
    TSS = sum((y - np.mean(y))**2)
    
    #yPredict = b[0] + b[1]*x
    yPredict = polyRegPredict(b,x)
    res = y - yPredict
    rss = res**2
    loss = np.sum(rss)
    
    R2 = 1 - (loss/TSS)

    return R2, b

def leaveOneOutCV(df, y, fitfnc = multiRegFit, predfnc = multiRegPredict, args =()):

    # first create an array that represent the index 
    ind = np.arange(len(df.index))

    N = len(df.index)

    yp_cv = np.zeros(N)

    # use np.array_split to generate indices for folds
    folds = np.array_split(ind, N)
    
    r,b0 = fitfnc(df, y, args)
    

    for f in np.arange(N): 
        folds_cp = folds.copy() # creating a copy of the folds array
        test_ind = folds[f] # get the indices for test set
        df_test  = df.loc[test_ind] # set one fold aside for testing


        del folds_cp[f]        # delete the test set indices
        train_ind = np.concatenate(folds_cp, axis = 0) # concatenate all the remaining indices into 1 array
        df_train  = df.loc[train_ind]
        ytrain = y.loc[train_ind]
        
        r,b = fitfnc(df_train, ytrain, args)
        yp_cv[test_ind] = predfnc(b, df_test, args)

    # TSS
    TSS = sum((y - y.mean())**2)

    # cross validated RSS
    RSScv = sum((y - yp_cv)**2)

    # cross validated R2
    R2cv = 1-RSScv/TSS

    # fit and predict
    yp = predfnc(b0, df, args)

    # 
    TSS = sum((y-y.mean())**2)
    RSS = sum((y-yp)**2)
    R2  = 1-RSS/TSS
    
    return R2cv, R2

## Lecture 9

### Multiple regression analysis
In multiple regression analysis the response varibale is explained by a linear combination of multiple explantory variables.
- each varable has its own regression coefficient (b)
- We alwasy have an intercept so number of parameters = number of explanatory variables + 1 
- Polynomial regression is a special form of multiple regression 
- In general, the response plane is a Q dimensional hyperplane in Q+1 hyperspace

|         | Ex 1  | Ex 2 | Ex 3 |
|---------|--------------------|---------------|-------|
| Visits    | 0.10 | 0.10  | 0.10 |
| Weeks | 0.30 | 0.30  | 0.30 |
| Visits + weeks | 0.29 | 0.33        | 0.40    |

**Ex1**. After accounting for length of pregnacy, the number of visits **do not add predictive value**  
**Ex2**. After accounting for length of pregnacy, the number of visits **do add predictive value** but there is some overlapping variance that both regressors can predict  
**Ex1**. They are not correlated. **They cannot explain the same part of the variance.**

The bootstrap distribution tells us what variability we should expect in the slope parameter, if we took a new random sample from the population.
If CI does not include 0 then it can be agreed that there is evidence of a positive influence.

Inference on models: We can determine which model is the most appropriate by comparing the (cross-vaildated) predicition performance.
- Comparison is at the level of models
- Using nested models, we can switch parameters on and off

Inference on parameters: Within a model, we can look at the uncertainty of parameters estimates using bootstrap
- Evaluation of uncertainty of a specific parameter in the context of the model
- We can construct CI usign bootstrap ad check if value of interest is within
If the model is good the inference on model and parameters should match.  

Positively correlated regressors lead to:
- negative correlation of slope estimates
- increase in the variability of the estimates 
Negatively correlated regressors lead to:
- positive correlation of slope estimates
- increase in the variability of the estimates 
Adding many correlated regressors increase teh uncertainty of all of them. R2cv will tell you what's most appropriate


In [ ]:
def multiRegPredict(b, df, xname):

    ## the intercept is the first element of the parameter array multiplied by 1.
    yp = np.ones(len(df.index)) * b[0]

    for i in range(len(xname)):
        xcurrent = df[xname[i]]
        yp = yp + b[i+1]*xcurrent # Add each regression value 

    return yp

def multiRegLossRSS(b, df, y, xname):
    # 1. calculate the residuals
    yp = multiRegPredict(b, df, xname)

    res = y - yp
    res2 = res ** 2
    RSS = sum(res2)
    
    
    # 1. initialize the derivative array
    deriv = np.zeros(len(b))

    ## the first element will be the derivative in respect to the intercept
    deriv[0] = -2*sum(res)
    
    # 2. build up the array using a for loop 
    for i in range(0, len(xname)):
        deriv[i+1] = -2*np.sum(df[xname[i]]*res)
    
    return RSS, deriv

def multiRegFit(df, y, xname):
    b0 = np.zeros(len(xname)+1)
    
    result = so.minimize(multiRegLossRSS, b0, args=(df, y, xname), jac=True)
    
    b = result.x
    
    TSS = sum((y - np.mean(y))**2)

    RSS, deriv = multiRegLossRSS(b,df,y,xname)
    R2 = 1 - RSS/TSS 

    return R2, b


def bootstrap(df, y, fitfcn, numIter = 500, args=()):

    R2, b = fitfcn(df,y,args)

    numParam = len(b)
    
    N = len(df.index)
    ind = np.arange(N)

    stat = np.zeros((numIter, numParam))
    
    for i in range (0, numIter):
        sample = np.random.choice(ind, N, replace = True)
        
        r2, stat[i, :] = fitfcn(df.iloc[sample], y[sample], args)
        
    return stat

## Lecture 10

### Logarithmic Regression
When coding categorical variables with more than one option, one group plays the role of the comparison group  

|         | X0  | X1 | X2 |
|---------|--------------------|---------------|-------|
| Canada    | 1 | 1 | 0 |
| USA | 1 | 0  | 1 |
| Mexico | 1 | 0 | 0  |

In this example Mexico is the comparison group. So the dummy variables (X1 and X2 are always 0). The intercept X0 is always 1.  
b0: mean(Mexico)  
b1: mean(Canada) - mean(Mexico)  
b2: mean(USA) - mean(Mexico). 

- Model-based comparison establishes which set of variables is teh most appropriate to explain the dependent variable. 
- Parameter bootstrap tells you whether the inlfuence of a variable is significant within the context of a specific model
- If the specific model that you are using is bad, your inference on the parameters will be bad (nonsensical, overfitting with a large CI)

Logistic Functions aim to fit probability values. 
1. Model the odds-ratio... the ratio of probability of an event occuring/not occuring **p/(1-p)**
2. Model the log of the odds-ratio **log(p/(1-p))**

Follows a 2 step modelling approach:
an = b0 + b1x
pn = 1 / (1 + exp(-an))

Loss funtion:
- Should captue the deviation of the observed values from prediction
- Can use squared or absolute error
- The statistically principled solution is to try to maximize the probability of the data under the model p(Data|parameters)
- p(Data|parameters) is called the likelihood
- The parameter values that maximize p(Data|parameters) are called the maximum-likelihood estimates

Likelihood: 
- Likelihood is the product of all the single probabilities (assuming independence of observations - coin toss)
- The likelihood when theres a lot of data will get very close to 0 and because it's a product taking deriitives are hard... this is solved by using log 

Log Loss:
- By maximizing the log-likelihood we maximize the likelihood
- By setting the loss (L) to the negative log-likelihod we can maximize the likelihood... this approach is called the Log-loss

### SEE PAPER CHEETSHEET FOR GRADIENT OF THE LOSS DERIVATIONS

Minimizing the RSS loss function maximizes the likelihood of y being normally distributed

|         | Linear Regression          | Logistic Regression     |
|:--:|:-------------------:|:-------------:|
| Prediction function    | linear fcn | logistic of linear fcn      |
| Likilihood | Gaussian | Bernulli  |
| Loss Function  | RSS: <br /> $(y-p)^2$ | Log-likeli/Log-loss: <br /> $-ylog(p)$ - $(1-y)log(1-p)$ | 
| Criterion used for CV | " |  " |
| Better model has  | Lower RSS, Higher R2| Higher log-likelihood | 


Interpreting log-likelihoods:
- Log probability of the data given the model p(Data|Model)
- Actual values do not mean much (depends on the scaling of the data), can be large negative or positive numbers
- Can only interpret the difference of log-likelihoods (log BF) or ratio of likelihoods (BF)

| Bayes Factor        | Log Bayes Factor |Evidence         |
|---------|----------|----------|
| <1    |  <0 | Negative, in favour of the other hypothesis |
| 1-3 | 0-1 | Barely worth mentioning |
| 3-20  | 1-3  | Positive              | 
| 20-150| 3-5  | Strong|
|>150| >5  | Very Strong|  

- The best fitting model/parameter is the one, for which the likelihood of the data given that model/paramter is the highest 
- Because the log is a monotonic function, we maximize the likelihod if we maximize the log-likelihood

In [ ]:
# ADD A DF COLUMN 

# with math
df['bodyL'] = df.apply(lambda x: x['totalL'] - x['tailL'], axis=1)

# discrete variable
popI = df['pop'] == 'Vic'
popI = np.double(popI)
df['popI']  = popI

# Graph with seperation between discrete variables
plt.scatter(df.age[df.sexI==0], df.bodyL[df.sexI==0])
plt.scatter(df.age[df.sexI==1], df.bodyL[df.sexI==1])
yp = multiRegPredict(b, df,["sexI", "age"])
plt.plot(df.age[df.sexI==0], yp[df.sexI==0])
plt.plot(df.age[df.sexI==1], yp[df.sexI==1])

# Logistic regression 

def logisticRegPredict(b, df, xname):
    
    # initialize the predicted array (starting from the intercept)
    an = np.ones(len(df.index)) * b[0]
    
    # get the predicted values
    for i in range(len(xname)):
        an = an + b[i + 1] * df[xname[i]]
        
    pn = 1 / (1 + np.exp(-an))
    
    return pn

def logisticRegLoss(b, df, y, xname):
    # 1. use the prediction function
    pn = logisticRegPredict(b, df, xname)

    # 2. get the cost:
    ## step by step implementation
    A = y*np.log(pn)
    B = (1-y)
    C = np.log(1 - pn)

    ## now look at the formula and implement it using A, B, C
    L = -1 * sum(A + B * C)

    # 3. get the derivatives array
    ## initialize with the value for intercept:
    deriv = np.zeros(len(xname)+1)
    res = y - pn
    
    deriv[0] = -sum(res) 

    ## now loop over and get the grad
    for i in range(len(xname)):
        deriv[i+1]=-np.sum(df[xname[i]]*res)
        
    return L, deriv # Returns loss (rss equiv) and b 

def logisticRegFit(df, y, xname, figure = True):
    N = len(xname)
    b0 = np.zeros(N+1)
    
    result = so.minimize(logisticRegLoss,b0,args=(df,y,xname),jac=True)
    
    b = result.x
    ll = -result.fun
    
    if (N == 1) & (figure ==True):
        plt.scatter(df[xname], y) 
        
        mini, maxi = min(df[xname[0]]), max(df[xname[0]])
        xp = np.arange(mini, maxi, (maxi - mini)/100)
        yp = b[1]*xp + b[0]
        pn = np.exp(yp)/(1+np.exp(yp))
        plt.plot(xp, pn, 'r-')
    
    
    return ll, b # returns log-loss (R2 equiv) and b 

## Lecture 11

Information Criteria
Instead of CV we can try to estimate the difference between fit and prediction performance
- Lower number indicate better model
- Differences in AIC/BIC can be interpreted by the bayes table (x2)

**Akaike Information Criterion**  
$ AIC = -2l + 2k $  
Because the log likelihood scales in number of data points (N), the penalty becomes nearly unimportant for large data sets

**Bayes Information Criterion**  
$ BIC = -2l + klog(N) $. 
The BIC scales the penalty softly, therefore preferrign slightly simpler models for larger data sets


Model Selection:
- When the model gets too complex, the predicted R2 goes down, even if you have the correct model (use fewer regressors)
- Reality is all models are wrong some are just useful
- Search for the best subset of regressors in a whole class of models 
- You could test all models and select the best on for an exhaustive search or use a restricted model search to find a good model without fitting all of them

Forward Model Search: 
- Start with the intercept and add one explanatory variable at a time in every combination. Only continue down the path that adds R2cv value. 
Backward Model Search:
- Start with the full model and drop one explanatory variable at a time in every combination. Only continue down the path that adds R2cv value.

Regularization:
- Is a way in which we can use many regressors
- Main idea is to ensure that the regression coefficients are small
- The easiest way to do this is to penalize the size of the regresson coeffiecients in the loss function 
- This makes the fit smoother
- Because of this, all regressors eed to be on a similar scale, so it is common practice to z-standardize all regressors (subtract mean and divide by SD)
    - B0 is not reglarized



In [ ]:
def KfoldCVmultReg(D,y,xname,K=20):

    N = len(y) #Number of observations
    yp= np.zeros(N)

    # Make an index vector with K folds
    ind = np.arange(N)
    ind = np.floor(ind/N*K)
    
    # Get overall model fit 
    R2,b_all=multRegFit(D,y,xname)

    # Loop over the crossvalidation folds 
    for i in range(K):
        r,b=multRegFit(D[ind!=i],y[ind!=i],xname,b0=b_all)
        yp[ind==i]=multRegPredict(b,D[ind==i],xname)
        
    # Calculate crossvalidated model fit 
    TSS  = sum((y-y.mean())**2)
    RSScv = sum((y-yp)**2)
    R2cv = 1-RSScv/TSS
    return R2cv,R2 

def zstandardize(d):
    d = (d-d.mean())/d.std()
    return d

def ridgeLoss(b,D,y,xname, a = 1):
    
    predY = multRegPredict(b,D,xname)
    res = y-predY
    rss = sum(res**2)
    grad=np.zeros(len(b))
    grad[0]=-2*np.sum(res)
    
    loss = rss + a*(np.sum(b[1:]**2))
    
    for i in range(len(xname)):
        grad[i+1]=-2*np.sum(D[xname[i]]*res) + (2 * a * b[i+1])
        
    return (loss,grad)

def ridgeFit(D,y,xname=[], a = 1, figure=0,b0=[]):
    
    k=len(xname)+1
    if (len(b0)!=k):
        b0=np.zeros((k,))
    RES = so.minimize(ridgeLoss,b0,args=(D,y,xname,a),jac=True)
    b=RES.x # Results
    res = y-np.mean(y)
    TSS = sum(res**2)
    
    predY = multRegPredict(b,D,xname)
    res = y-predY
    RSS = sum(res**2)
    
    #RSS,deriv = multRegLossRSS(b,D,y,xname)
    R2 = 1-RSS/TSS 
    if (k==2 and figure==1):
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.scatter(D[xname[0]],y)
        xRange=[min(D[xname[0]]),max(D[xname[0]])]
        xp=np.arange(xRange[0],xRange[1],(xRange[1]-xRange[0])/50)
        yp=b[0]+b[1]*xp
        ax.plot(xp,yp,'r-')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    return (R2,b)


def KfoldCVridge(D, y, xname, K = 20, fitfcn = ridgeFit, param={}, predfcn = multRegPredict, a = 1):

    # first create an array that represent the index 
    ind = np.arange(len(df.index))
    
    yp_cv = np.zeros(len(df.index))

    # use np.array_split to generate indices for folds
    folds = np.array_split(ind, K)
    
    N = len(folds)
    
    r,b0 = fitfcn(D,y,xname,a,**param)
    
    for f in np.arange(N): 
        folds_cp = folds.copy() # creating a copy of the folds array
        test_ind = folds[f] # get the indices for test set
        df_test  = df.loc[test_ind] # set one fold aside for testing


        del folds_cp[f]        # delete the test set indices
        train_ind = np.concatenate(folds_cp, axis = 0) # concatenate all the remaining indices into 1 array
        df_train  = df.loc[train_ind]
        ytrain = y.loc[train_ind]
        
        r,b = fitfcn(df_train,ytrain,xname,a, **param) # multRegFit(D,y,xname=[],figure=0,b0=[])
        yp_cv[test_ind] = predfcn(b, df_test, xname)# **param) # multRegPredict(b,D,xname)
        

    # TSS
    TSS = sum((y - y.mean())**2)

    # cross validated RSS
    RSScv = sum((y - yp_cv)**2)

    # cross validated R2
    R2cv = 1-RSScv/TSS

    # fit and predict
    yp = predfcn(b0, D, xname) # **param)

    # 
    TSS = sum((y-y.mean())**2)
    RSS = sum((y-yp)**2)
    R2  = 1-RSS/TSS
    
    return R2cv, R2
 

### Modified Log functions - Not coded by me
Improvements
1. prevent log(0) errors by making sure that your predicted value never is smaller than 1e-20 or larger than 1-1e-20. (tip you can use the numpy function `clip`)

2. Let logisticRegFit take an additional input parameter, telling it whether it should plot a figure or not (figure=1) 

3. Let logisticRegFit take an additional input parameter, specifying the starting value for the parameters (b0=[]). If b0 is empty, the function should start with a vector off all zeros.

In [ ]:
#MODIFIED LOG FUNCTIONS 

def logisticRegPredict(b,D,xname):
    yp=np.ones(len(D.index))*b[0]       # Start out with the intercept  
    for i in range(len(xname)):
        yp=yp+D[xname[i]]*b[i+1]        # Add the prediction of each regressor seperately 
    p = np.exp(yp)/(1+np.exp(yp))
    p = p.clip(1e-12,1-(1e-12))
    return p 
    
def logisticRegLoss(b,D,y,xname):
    p = logisticRegPredict(b,D,xname)
    cost = -y*np.log(p)-(1-y)*np.log(1-p)
    N=len(xname)
    grad=np.zeros(N+1)
    res = y-p
    grad[0]=-sum(res)
    for i in range(N):
        grad[i+1]=-np.sum(D[xname[i]]*res)         # Add each regressor 
    return (cost.sum(),grad)
    
def logisticRegFit(D,y,xname,figure=0,b0=[]):
    k=len(xname)+1
    if (len(b0)!=k):
        b0=np.zeros(k)
    RES = so.minimize(logisticRegLoss,b0,args=(D,y,xname),jac=True)
    b = RES.x
    ll = -RES.fun # Negative function value is the log likelihood 
    p = logisticRegPredict(b,D,xname)
    if (k==2 & figure==1):
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.scatter(D[xname[0]],y)
        xRange=[min(D[xname[0]]),max(D[xname[0]])]
        xp=np.arange(xRange[0],xRange[1],(xRange[1]-xRange[0])/50)
        yp=b[0]+b[1]*xp
        pp=np.exp(yp)/(1+np.exp(yp))
        ax.plot(xp,pp,'r-')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    return (ll,b)


def KfoldCVlogisticReg(D,y,xname,K=20,fitfcn=logisticRegFit,predictfcn=logisticRegPredict):
    N = len(y) #Number of observations
    yp= np.zeros(N)
    ind = np.arange(N)
    ind = np.floor(ind/N*K)
    
    # Get overall model fit 
    LL,b_all=fitfcn(D,y,xname,figure=0)
    
    # Loop over the crossvalidation folds 
    for i in range(K):
        r,b=fitfcn(D[ind!=i],y[ind!=i],xname,b0=b_all,figure=0)
        yp[ind==i]=predictfcn(b,D[ind==i],xname)
    LLcv = sum(y*np.log(yp)+(1-y)*np.log(1-yp))
    return LLcv,LL 